In [2]:
# Imports
import xarray as xr
import numpy as np, matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from datetime import datetime, timedelta
import pandas as pd
import seaborn as sns
import matplotlib as mpl

# Figure settings
mpl.rcParams['figure.dpi'] = 200
plt.rcParams.update({'font.size': 12})

# Path to data locations
CN_path = '/nird/projects/NS9600K/sldam/data/cloudnet_data/20191112_ny-alesund_classification.nc'

HB1_path = '/nird/projects/NS9600K/sldam/data/nascent_data/HoloBalloon/HoloBalloon_t30/191112_1_t30.nc'
HB2_path = '/nird/projects/NS9600K/sldam/data/nascent_data/HoloBalloon/HoloBalloon_t30/191112_2_t30.nc'
HB3_path = '/nird/projects/NS9600K/sldam/data/nascent_data/HoloBalloon/HoloBalloon_t30/191112_3_t30.nc'

MD3_processed_path = '/nird/projects/NS9600K/sldam/data/my_processed_data/MD3_HighRes_12Nov_IWF_CT_winddir.nc' # 1-min, IWF, wind, CT, full 12 Nov
MA3_processed_path = '/nird/projects/NS9600K/sldam/data/my_processed_data/MA3_HighRes_12Nov_IWF_CT_winddir.nc' # 1-min, IWF, wind, CT, full 12 Nov
MAS3_processed_path = '/nird/projects/NS9600K/sldam/data/my_processed_data/MAS3_HighRes_12Nov_IWF_CT_winddir.nc' # 1-min, IWF, wind, CT, full 12 Nov


# Aug 30, 24: New colors tested against color blindness  simulator
model_colors = ['#6E5CBB', '#8DC3DC', '#A64A36', '#CC793B', '#F1B872']
colors = {'Default': '#A64A36', 'Adapted': '#CC793B', 'Adapted+SIP': '#F1B872', 'CloudNet': '#8DC3DC', 'HoloBalloon': '#6E5CBB'}
phase_colors = ['#3188B9', '#5BB558', '#ACADAC']
phase_labels = ['liquid', 'mixed', 'ice']

In [3]:
# Helper functions for importing and processing the data to prepare and simplify analysis

def get_CN(datapath):
    '''
    Import the Cloudnet classification product and rename the classes relevant here

    Arguments:
    datapath  (str)            - path to location where data is stored
    Returns:
    to_ret    (xarray Dataset) - time-height xarray Dataset field with classified cloud according to Cloudnet         
    '''

    CN = xr.open_dataset(datapath).isel(height=slice(None, 350)).target_classification
    CN = xr.where((CN != 1) & (CN != 4) & (CN != 5), x=np.nan, y=CN)

    to_ret = xr.where(CN==4, x=2, y=CN) # Ice
    to_ret = xr.where(CN==1, x=0, y=to_ret) # Liquid
    to_ret = xr.where(CN==5, x=1, y=to_ret) # Mixed
    return to_ret

def get_HB(datapath, tslice, threshold=0.1):
    '''
    Import the HoloBalloon ice water fraction data, convert time format, and classify

    Arguments:
    datapath  (str)            - path to location where data is stored
    tslice    (slice)          - slice object of the timesteps to use for analysis. This needs to be specified manually to exclude ascent/descent
    threshold (float)          - IWF thresholds used to distingish the mixed-phase zone. Default is 0.1, following Korolev et al. (2003). 
    Returns:
    to_ret    (xarray Dataset) - xarray Dataarray timeseries of classified cloud according to HoloBalloon         
    '''
    HB = xr.open_dataset(datapath).isel(time=tslice)

    times = HB.time.to_numpy()
    t = np.array([datetime.fromordinal(int(t) - 366) + timedelta(days=t%1) for t in times])
    HB['time'] = t

    IWF = HB.Ice_content / HB.Total_content
    classified = xr.where(IWF > 1 - threshold, x=2, y=IWF)
    classified = xr.where(IWF < threshold, x = 0, y=classified)
    to_ret = xr.where((IWF < 1 - threshold) & (IWF > threshold), x=1, y=classified)
    return to_ret


def get_processed_data(datapath, threshold=0.1):
    '''
    Import the WRF simulation preprocessed data and classify

    Arguments:
    datapath       (str)            - path to location where data is stored
    threshold      (float)          - IWF thresholds used to distingish the mixed-phase zone. Default is 0.1, following Korolev et al. (2003). 
    Returns:
    IWF_classified (xarray Dataset) - xarray Dataset with time/height field of classified cloud         
    '''
    x, y = 43, 50
    data = xr.open_dataset(datapath).IWF.isel(west_east=x, south_north=y)
    return classify_IWF(data, threshold=threshold)
 
def classify_IWF(IWF, threshold=0.1):
    '''Classify phase based on given IWF thresholds. Default is 0.1 and 0.9, following Korolev et al. (2003)'''
    IWF_classified = xr.where((IWF < 1-threshold) & (IWF > threshold), x=0.5, y=IWF) # mixed-phase
    IWF_classified = xr.where(IWF > 1-threshold, x=1, y=IWF_classified) # ice
    IWF_classified = xr.where(IWF < threshold, x=0, y=IWF_classified).T * 2 # liquid
    return IWF_classified

In [4]:
limit_time = True # Option to use only data from when holoballoon was flying
threshold = 0.1   # Threshold for IWF to use when classifying liquid/mixed/ice.
subsample = 2     # Used only for CN and HB

#Model levels (found by wrf.getvar(nc.Dataset(MAS3_path), 'z').isel(bottom_top=slice(0, 85), west_east=50, south_north=50).to_numpy())
h = np.array([11.3408785,   34.04713,   56.80371,   79.61095, 102.47005,  125.38474,  148.3563 ,  171.38313,
    194.46507,  217.6024 ,  240.79466,  264.04178, 287.3438 ,  310.70056,  334.11267,  357.57974,
    381.10104,  404.67712,  428.30814,  451.99432, 475.73615,  499.5331 ,  523.38495,  547.292  ,
    571.2547 ,  595.27405,  619.34924,  643.4812 , 667.672  ,  691.9206 ,  716.2291 ,  740.60077,
    765.0361 ,  789.5347 ,  814.0979 ,  838.726  , 863.419  ,  888.1784 ,  913.0036 ,  937.8924 ,
    962.8429 ,  987.85596, 1012.93317, 1038.0742 , 1063.2799 , 1088.5518 , 1113.89   , 1139.2958 ,
    1164.7697 , 1190.3134 , 1220.2161 , 1254.5106 , 1288.9487 , 1323.539  , 1358.2928 , 1393.2336 ,
    1428.377  , 1463.725  , 1499.282  , 1535.0516 , 1571.0369 , 1607.2264 , 1643.6042 , 1680.1637 ,
    1716.9023 , 1753.8174 , 1790.9053 , 1828.1632 , 1865.5887 , 1903.1819 , 1940.9419 , 1978.8679 ,
    2016.9591 , 2055.214  , 2093.6353 , 2137.0706 , 2185.5688 , 2234.331  , 2283.3591 , 2332.6555 ,
    2382.2231 , 2432.0654 , 2482.1836 , 2532.5793 , 2583.2563])

#Accessing HoloBalloon data
HB_tslices = [slice(20, 136), slice(32, 191), slice(30, 274)]

HB_orig = xr.concat([get_HB(path, tslice, threshold=threshold) for path, tslice in zip((HB1_path, HB2_path, HB3_path), HB_tslices)], dim='time')
HB_red = HB_orig.isel(time=slice(None, None, subsample))

HB_heights = xr.concat([xr.open_dataset(path).isel(time=tslice).instData_height for path, tslice in zip((HB1_path, HB2_path, HB3_path), HB_tslices)], dim='time')


tslice_M = slice(600, 1021)
tslice_CN = slice('2019-11-12T09:59:45', '2019-11-12T17:00:20')

CN_orig = get_CN(CN_path).sel(time=tslice_CN)
CN_red = CN_orig.isel(time=slice(None, None, subsample))

MD3 = get_processed_data(MD3_processed_path, threshold=threshold).rename({'bottom_top': 'height', 'Time': 'time'}).isel(time=tslice_M)
MA3 = get_processed_data(MA3_processed_path, threshold=threshold).rename({'bottom_top': 'height', 'Time': 'time'}).isel(time=tslice_M)
MAS3 = get_processed_data(MAS3_processed_path, threshold=threshold).rename({'bottom_top': 'height', 'Time': 'time'}).isel(time=tslice_M)


In [5]:
def add_arrays(a, b):
    # Function to add together the counted pocket lengths for each of the layers
    if type(a) == int:
        return b
    result_inds = np.unique(list(a[0]) + list(b[0]))
    result_counts = []

    current_ai = 0
    current_bi = 0

    for i in result_inds:
        if i in a[0] and i in b[0]:
            result_counts.append(a[1][current_ai] + b[1][current_bi])
            current_ai += 1
            current_bi += 1
        
        elif i in a[0]:
            result_counts.append(a[1][current_ai])
            current_ai += 1

        elif i in b[0]:
            result_counts.append(b[1][current_bi])
            current_bi += 1

        else:
            print('Fatal error')

    return np.array([result_inds, result_counts])

def select_layer(input, n=0, reference_level=None):
    # select nth layer counting from reference_level or top
    if reference_level is None: # Assume cloud top as reference level
        hc = xr.where(input >= 0, x=input.height, y=0)
        reference_level = hc.argmax(dim='height')
    
    return input.isel(height=reference_level - n)
    
def select_layers(input, n=2, reference_level=None):
    # Select n layers counting from cloud top or reference layer down
    total = select_layer(input, n=0, reference_level=reference_level)
    for i in range(1, n):
        total = xr.concat([total, select_layer(input, i, reference_level)], dim='y')
    return total.rename({'y': 'height'})

def compute_segment_lengths(data, target_number):
    # Find consecutive segments containing the target number (0 liquid, 1 mixed, 2 ice)
    consecutive_segments = np.split(data, np.where(np.diff(np.concatenate(([0], data == target_number, [0]))) != 0)[0])

    # Remove empty segments and count lengths
    segment_lengths = [len(seg) for seg in consecutive_segments if len(seg) > 0 and seg[0] == target_number]

    if segment_lengths == []:
        lengths, counts = [1], [0]
    
    else:
        # Count occurrences of each segment length
        length_counts = {length: segment_lengths.count(length) for length in set(segment_lengths)}

        lengths, counts = zip(*sorted(length_counts.items()))
    return np.array([lengths, counts])

def find_min_max(array1, array2, operator=np.max):
    '''Keep the max or min counts for each index in two sets of indices and counts'''
    
    try:
        combined_array = np.column_stack((array1, array2)) # Stack arrays together
    except ValueError:
        array1 = np.array([[], []])
        combined_array = np.column_stack((array1, array2))

    unique_indices = np.unique(combined_array[0]) # Keep only unique indices

    result = np.zeros_like(unique_indices, dtype=int) # Initialize resulting array to fill


    for idx in unique_indices: # loop through unique_indiced and keep max/min counts
        index_mask = combined_array[0] == idx
        result[np.where(unique_indices == idx)] = operator(combined_array[1, index_mask])

    return np.array([unique_indices, result], int)

def mean_median_length(segment_lengths, stat='median'):
        '''Find the mean or median segment length of input [[lengths, counts]]'''
        if stat == 'median':
            return np.median(np.repeat(segment_lengths[0], segment_lengths[1]))
        elif stat =='mean':
            return np.mean(np.repeat(segment_lengths[0], segment_lengths[1]))
        else:
            raise ValueError('stat must be either median or mean')
        
def median_length(segment_lengths):
    if len(segment_lengths) == 3: # All phases
        tot = add_arrays(add_arrays(segment_lengths[0], segment_lengths[1]), segment_lengths[2])
        mean = np.median(np.repeat(tot[0], tot[1]))
    else: # Single phase entered
        mean = np.median(np.repeat(segment_lengths[0], segment_lengths[1]))

    return mean

def mean_length(segment_lengths):
    if len(segment_lengths) == 3: # All phases
        tot = add_arrays(add_arrays(segment_lengths[0], segment_lengths[1]), segment_lengths[2])
        mean = np.mean(np.repeat(tot[0], tot[1]))
    else: # Single phase entered
        mean = np.mean(np.repeat(segment_lengths[0], segment_lengths[1]))

    return mean

def repmat(arr):
    return [np.repeat(a[0], a[1]) for a in arr]

def continuous_indices(segment_lengths):
    continuous_segment_lengths = []
    for phase in segment_lengths:
        indices, counts = phase
        
        new_indices = np.arange(1, 3000)
        index_to_count = dict(zip(indices, counts))
        new_counts = np.array([index_to_count.get(i, 0) for i in new_indices])
        continuous_segment_lengths.append([new_indices, new_counts])
    return continuous_segment_lengths



def get_segment_lengths(data, n_layers, reference_level=None, continuous=True):
    segment_lengths = [[], [], []]

    for i in range(3):
        TOT = 0
        if n_layers != 1:
            for n in range(n_layers):
                seg_lens = compute_segment_lengths(select_layer(data, n=n, reference_level=reference_level), target_number=i)
                TOT = add_arrays(TOT, seg_lens)
        else:
            TOT = compute_segment_lengths(data, target_number=i)
        segment_lengths[i] = TOT
    if continuous:
        return continuous_indices(segment_lengths)
    else:
        return segment_lengths
    
def SHS(segment_lengths):
    '''Spatial Heterogeneity Score based on dAllesandro etal (2021)'''
    try: # All phases
        n_samples = sum([sum(x[0] * x[1]) for x in segment_lengths])
        n_segments = sum([sum(x[1]) for x in segment_lengths])
    except TypeError: # Single phase entered
        n_samples = sum(segment_lengths[0] * segment_lengths[1])
        n_segments = sum(segment_lengths[1])

    return (n_samples - n_segments) / (n_samples - 1)


def plot_heterogeneity(data, ax, title, km_conversion=1, percent=True, stat='median'):
    ''' Plot the heterogeneity plot from given pocket length counts per phase.
        data should be for format [liquid, mixed, ice] where each of liquid/mixed/ice are 2d-arrays with
        pocket lengths and counts.'''

    TOTAL_N_POCKETS = np.sum([np.sum(phase[1]) for phase in data])
    textstring = 'Pocket occurrence: \n'
    pc = 1
    if percent:
        pc = 100
    
    for i, phase in enumerate(data):
        ax.plot(phase[0] * km_conversion, phase[1] / TOTAL_N_POCKETS * pc, color=phase_colors[i], label=phase_labels[i], linewidth=2)
        ax.axvline(mean_median_length(phase, stat=stat) * km_conversion, linestyle='dashed', color=phase_colors[i], alpha=0.5)

        textstring += f'{phase_labels[i]}={round(sum(phase[1]) / TOTAL_N_POCKETS * 100, 1)}%\n'
    ax.axvline(-100, linestyle='dashed', color='black', label=stat)

    ax.text(0.97, 0.979, textstring, transform=ax.transAxes,
        verticalalignment='top', horizontalalignment='right',
        bbox=dict(facecolor='white', edgecolor='none'))
    
    ax.set_title(title)

In [6]:
# Cloud interior

fig, ax = plt.subplots(1, 4, figsize=(10, 3), sharey=True, sharex=True)
colors = {'CloudNet 30s': '#8DC3DC', 'HoloBalloon 30s': '#6E5CBB', 'CloudNet 1min': '#8DC3DC', 'HoloBalloon 1min': '#6E5CBB'}


C = [L * 0.3 for L in repmat(continuous_indices(get_segment_lengths(CN_orig, n_layers=30, reference_level=137)))]
H = [L * 0.3 for L in repmat(get_segment_lengths(HB_orig, n_layers=1))]

liquid_df = pd.DataFrame([C[0], H[0]], index=['CloudNet 30s', 'HoloBalloon 30s']).T.assign(phase='liquid')
mixed_df = pd.DataFrame([C[1], H[1]], index=['CloudNet 30s', 'HoloBalloon 30s']).T.assign(phase='mixed')
ice_df = pd.DataFrame([C[2], H[2]], index=['CloudNet 30s', 'HoloBalloon 30s']).T.assign(phase='ice')
total_df = pd.DataFrame([np.concatenate([e[0], e[1], e[2]]) for e in [C, H]], index=['CloudNet 30s', 'HoloBalloon 30s']).T.assign(phase='total')

stat = 'proportion'

sns.ecdfplot(liquid_df, ax=ax[0], palette=colors, legend=False, stat=stat)
sns.ecdfplot(mixed_df, ax=ax[1], palette=colors, legend=False, stat=stat)
sns.ecdfplot(ice_df, ax=ax[2], palette=colors, legend=False, stat=stat)
sns.ecdfplot(total_df, ax=ax[3], palette=colors, legend=False, stat=stat)




# Reduced
C = [L * 0.6 for L in repmat(continuous_indices(get_segment_lengths(CN_red, n_layers=30, reference_level=137)))]
H = [L * 0.6 for L in repmat(get_segment_lengths(HB_red, n_layers=1))]

liquid_df = pd.DataFrame([C[0], H[0]], index=['CloudNet 1min', 'HoloBalloon 1min']).T.assign(phase='liquid')
mixed_df = pd.DataFrame([C[1], H[1]], index=['CloudNet 1min', 'HoloBalloon 1min']).T.assign(phase='mixed')
ice_df = pd.DataFrame([C[2], H[2]], index=['CloudNet 1min', 'HoloBalloon 1min']).T.assign(phase='ice')
total_df = pd.DataFrame([np.concatenate([e[0], e[1], e[2]]) for e in [C, H]], index=['CloudNet 1min', 'HoloBalloon 1min']).T.assign(phase='total')


sns.ecdfplot(liquid_df, ax=ax[0], palette=colors, legend=False, stat=stat, linestyle='dashed')
sns.ecdfplot(mixed_df, ax=ax[1], palette=colors, legend=False, stat=stat, linestyle='dashed')
sns.ecdfplot(ice_df, ax=ax[2], palette=colors, legend=False, stat=stat, linestyle='dashed')
sns.ecdfplot(total_df, ax=ax[3], palette=colors, legend=False, stat=stat, linestyle='dashed')

#sns.move_legend(ax[2], 'center right', bbox_to_anchor=(2, 0.6), frameon=False)
#sns.move_legend(ax[3], 'center right', bbox_to_anchor=(2, 0.4), frameon=False)

dummyax = ax[3].twinx()
dummyax.plot([100000, 100001], [100000, 100001], color='#8DC3DC', linestyle='solid', label='Cloudnet 30s')
dummyax.plot([100000, 100001], [100000, 100001], color='#8DC3DC', linestyle='dashed', label='Cloudnet 1min')
dummyax.plot([100000, 100001], [100000, 100001], color='#6E5CBB', linestyle='solid', label='HoloBalloon 30s')
dummyax.plot([100000, 100001], [100000, 100001], color='#6E5CBB', linestyle='dashed', label='HoloBalloon 1min')

dummyax.axis('off')
dummyax.legend(frameon=False, loc='center right', bbox_to_anchor=(2.2, 0.5))

plt.xscale('log')
#plt.ylim(0, 1.03)
ax[0].set_title('Liquid')
ax[1].set_title('Mixed')
ax[2].set_title('Ice')
ax[3].set_title('Total Cloud', y=1.)
ax[3].set_xlim(0.2, 100)
ax[0].set_ylabel('CDF')
#[x.set_xlabel('pocket length (km)') for x in ax]
ax[1].set_xlabel('Pocket Length (km)')
ax[1].xaxis.set_label_coords(1, -0.16)

ax[0].text(0.96, 0.096, 'A', transform=ax[0].transAxes, fontsize=14, 
            va='top', ha='right')

ax[1].text(0.96, 0.096, 'B', transform=ax[1].transAxes, fontsize=14, 
            va='top', ha='right')

ax[2].text(0.96, 0.096, 'C', transform=ax[2].transAxes, fontsize=14, 
            va='top', ha='right')

ax[3].text(0.96, 0.096, 'D', transform=ax[3].transAxes, fontsize=14, 
            va='top', ha='right')


plt.subplots_adjust(wspace=0)
plt.show()